# Importing Required Packages

In [1]:
import wolframalpha
import pyttsx3
import speech_recognition as sr
import datetime
import wikipedia
import numpy as np
import face_recognition
import os
import cv2, time, pandas
from datetime import datetime
import csv
import sys
import winsound
from selenium import webdriver
from csv import reader
import time

### Data Base

In [2]:
#Link Database
import mysql.connector

mydb = mysql.connector.connect(
  host="db4free.net",
  user="shibinthomas",
  password="shibint85@",
  database="shibinthomas"
)

mycursor = mydb.cursor()

In [3]:
#Print Database


### Function to open Object Detection

In [4]:
def objdet():
    cap = cv2.VideoCapture(0)  # open camera
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    while cap.isOpened():
        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=3)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #cv2.drawContours (frame1, contours, -1, (0,255,0), 2)
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if cv2.contourArea(contour) < 10000:
                continue
            cv2.rectangle(frame1, (x, y), (x + y, y + h), (0, 255, 0), 1)
            cv2.putText(frame1, "Status: Movement", (10, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1, )
            winsound.Beep(2500,50)
            #waobjdet()
        cv2.imshow("feed", frame1)   
        frame1 = frame2
        ret, frame2 = cap.read()
        key = cv2.waitKey(50)
        if key == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

## Bin Bot main Function

In [5]:
def takeCommand():
    r = sr.Recognizer() 
    with sr.Microphone() as source: 
        print('Listening...',end=" ") 
        r.pause_threshold = 0.7
        audio = r.listen(source)
        try: 
            print("Recognizing...") 
            Query = r.recognize_google(audio, language='en-in') 
            print("Recognized as: ", Query)
        except Exception as e:
            print(e)
            print("Please be more clear")
            return "none"
        return Query

def speak(audio):
    strlen=len(audio.split())
    if strlen>=20:
        audio=audio.split()[:20]
        audio=' '.join(map(str, audio)) 
    #print(audio)
    #waspeak(audio)
    engine = pyttsx3.init() 
    voices = engine.getProperty('voices') 
    engine.setProperty('voice', voices[0].id) 
    engine.say(audio) 
    engine.runAndWait()
    
def talking_tom():
    while(True):
        talk=takeCommand().lower()
        if "none" in talk:
            continue
        if "exit" in talk:
            speak("Thank you for spending your time with me.")
            bin()
            break
        speak(talk)

def tellDay(): 
    day = datetime.today().weekday() + 1 
    Day_dict = {1: 'Monday', 2: 'Tuesday',3: 'Wednesday', 4: 'Thursday',5: 'Friday', 6: 'Saturday',7: 'Sunday'} 
    if day in Day_dict.keys(): 
        day_of_the_week = Day_dict[day] 
        print(day_of_the_week) 
        speak("The day is " + day_of_the_week) 


def tellTime(): 
    time = str(datetime.now()) 
    print(time) 
    hour = time[11:13] 
    min = time[14:16] 
    speak("The time is sir" + hour + "Hours and" + min + "Minutes")	 

    
def Hello(): 
    speak("hello sir I am your desktop assistant.Tell me how may I help you") 
    
                
def most_frequent(List):
    return max(set(List), key = List.count)

def bin():
    while(True):
        query = takeCommand().lower()
        if query=="none":
            continue
        elif "hello" in query:
            Hello()
        elif "bhai" in query:
            speak("Thank you i will take a leave")
            Take_query()
            break
        elif "bye" in query:
            speak("Thank you i will take a leave")
            Take_query()
            break
        elif "from wikipedia" in query: 
            speak("Checking the wikipedia ") 
            query = query.replace("wikipedia", "")  
            result = wikipedia.summary(query, sentences=1) 
            speak("According to wikipedia") 
            speak(result)
        elif "talking tom" in query:
            speak("Loading Talking Tom please wait")
            talking_tom()
            break
        else:
            try:
                res=client.query(query)
                output=next(res.results).text
                speak(output)
            except:
                speak("Please be more clear")

## Attendance Check and append

In [6]:
def check(name_list):
    if len(name_list)%100==0 and most_frequent(name_list)!="Unknown":
        std_name=most_frequent(name_list)
        print("Detected as:",std_name)
        speak('detected as'+std_name)
        while True:
            check=takeCommand().lower()
            if check=="yes":   
                mycursor.execute("SELECT * FROM userpage_userdetails where empId='%s'" % std_name),
                userDet = mycursor.fetchall()
                if(len(userDet)>0):
                    forKey=userDet[0][0]
                else:
                    forKey=0
                now=datetime.now()
                mycursor.execute("SELECT * FROM trackingDetails_trackers where empId='%s'" % std_name),
                myresult = mycursor.fetchall()
                empLength=len(myresult)
                print(empLength)
                flag=True
                if empLength%2==0:
                    flag=True
                else:
                    flag=False
                sql = "INSERT INTO trackingDetails_trackers (empId,is_active,lastUpdatedBy,forKey) VALUES (%s, %s,%s,%s)"
                val = (std_name,flag,datetime.now(),forKey)
                mycursor.execute(sql, val)
                mydb.commit()
                print(std_name)
                #markAttendance(std_name)
                print("Thank You Next Person please")
                speak("Thank You Next Person please")
                name_list.clear()
                #wacall(std_name)
            elif check=='quit':
                name_list.clear()
                speak('Exiting Attendance system')
                cv2.destroyAllWindows()
                Take_query()
            elif check=="no":
                print("Please try again")
                name_list.clear()
            if check=="yes" or check=="no" or check=="quit":
                break

## Face encoding Function

In [7]:
path='E:/Projects/Python/Face-Recogntion/photos1'
images=[]
classNames=[]
allList=[]
regNo=[]
myList=os.listdir(path)
for name in myList:
    curImg=cv2.imread(f'{path}/{name}')
    images.append(curImg)
    all=name
    name=name.split('_')
    reg=name[0]
    name=name[1]
    regNo.append(os.path.splitext(reg)[0])
    classNames.append(os.path.splitext(name)[0])
    allList.append(os.path.splitext(all)[0])
def findEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


encodeListKnown=findEncodings(images)
speak("Completed Encoding")
print("Completed Encoding")

known_face_encodings=encodeListKnown
known_face_names=classNames
print(known_face_names)

Completed Encoding
['EMP1', 'EMP2', 'EMP3', 'EMP4']


## Face Recognition Function

In [8]:
def face_rec():
    # Initialize some variables
    video_capture = cv2.VideoCapture(0)
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    # Csv file
    df = pandas.DataFrame(columns=["Start", "End"])
    count="Unknown"
    name_list=[]
    
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        status = 0
        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            # face_locations = face_recognition.face_locations(rgb_small_frame, number_of_times_to_upsample=0, model="cnn")
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # # If a match was found in known_face_encodings, just use the first one.
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                #name check
                face_names.append(name)                 
                if count!=name:
                    count=name
                    #speak(name)                            #--> To speak the detected name and the change in name

        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            status = 1
            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 0), 1)

            # Draw a label with a name below the face
            cv2.rectangle(
                frame, (left, bottom - 35), (right, bottom), (0, 0, 0), cv2.FILLED
            )
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            name_list.append(name)
            winsound.Beep(2500,50)
            
            # Check Function to check whether the person is right and Append to CSV
            check(name_list)
        # Display the resulting image
        cv2.imshow("Video", frame)


        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

## Main call Function

In [9]:
client = wolframalpha.Client('7E2K87-L6AH35EQ44')
def Take_query(): 
    while(True):
        query = takeCommand().lower()
        if "bin" in query:
            speak("Hello i am bin bot, here to help you")
            bin()
        elif "attendance" in query:
            speak("Loading attendance")
            face_rec()
        elif "detect" in query:
            speak("Loading Object Detection")
            objdet()
        elif "mask" in query:
            speak("Loading Mask Detection")
            mask()
        elif "exit" in query:
            speak("Turning off Bin bot, Thankyou")
            sys.exit()
            break
Take_query()

Listening... 

KeyboardInterrupt: 

<h1>Cheet Sheet</h1>
<h3>Sleep mode</h3>

1. **Bin** - Go to Bin mode
2. **Attendance** - Go to attendance mode
3. **detect** - Object Detection
4. **exit** - Switch off bin bot

<h3>Bin bot</h3>

1. **bye** - Go back to Sleep mode
2. **from wikipedia** - Search from Wikipedia
3. **Talking tom** - Switch to talking tom mode

<h3>Attendance</h3>

1. **yes** - To append Attendance
2. **no** - To try again
3. **quit** - Go back to sleep mode

In [11]:
sql = "INSERT INTO trackingDetails_trackers (empId,is_active,lastUpdatedBy) VALUES (%s, %s,%s)"
val = ("helldsssssssssssssssssssssso",True,datetime.now())
mycursor.execute(sql, val)
mydb.commit()

In [15]:
#Link Database
import mysql.connector

mydb = mysql.connector.connect(
  host="db4free.net",
  user="shibinthomas",
  password="shibint85@",
  database="shibinthomas"
)

mycursor = mydb.cursor()

In [17]:
#Print Database
mycursor.execute("SELECT * FROM trackingDetails_trackers")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(1, 'John', datetime.datetime(2022, 4, 15, 21, 57, 24, 281786), 1)
(2, 'John', datetime.datetime(2022, 4, 15, 22, 11, 6, 516091), 1)
(3, 'John', datetime.datetime(2022, 4, 15, 22, 11, 29, 884335), 1)
(4, 'John', datetime.datetime(2022, 4, 15, 22, 11, 51, 384052), 1)
(5, 'hello', datetime.datetime(2022, 4, 15, 22, 11, 56, 962446), 1)
(6, 'hello', datetime.datetime(2022, 4, 15, 22, 12, 12, 674127), 1)
(7, 'hello', datetime.datetime(2022, 4, 15, 22, 12, 32, 147008), 1)
(8, 'hello', datetime.datetime(2022, 4, 15, 22, 20, 47, 747382), 1)
(9, 'hello', datetime.datetime(2022, 4, 15, 22, 35, 14, 472968), 1)
(12, 'John', datetime.datetime(2022, 4, 15, 22, 48, 34, 119004), 1)
(13, 'Johcccccccccccccccn', datetime.datetime(2022, 4, 15, 22, 48, 42, 680769), 1)
(14, 'helldsssssssssssssssssssssso', datetime.datetime(2022, 4, 15, 22, 49, 15, 66616), 1)
(15, 'Saktheeswaran', datetime.datetime(2022, 4, 15, 22, 50, 37, 672644), 1)
(16, 'helldsssssssssssssssssssssso', datetime.datetime(2022, 4, 15, 22, 51

In [21]:
face_rec()

Detected as: EMP1
Listening... Recognizing...
Recognized as:  yes
4
EMP1
Thank You Next Person please
Detected as: EMP1
Listening... Recognizing...
Recognized as:  yes
5
EMP1
Thank You Next Person please


In [23]:
pip install beep

Note: you may need to restart the kernel to use updated packages.Collecting beep


ERROR: Could not find a version that satisfies the requirement scikit-learn==1.0.2 (from beep) (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0b1, 0.15.0b2, 0.15.0, 0.15.1, 0.15.2, 0.16b1, 0.16.0, 0.16.1, 0.17b1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19b2, 0.19.0, 0.19.1, 0.19.2, 0.20rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21rc2, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.22rc2.post1, 0.22rc3, 0.22, 0.22.1, 0.22.2, 0.22.2.post1, 0.23.0rc1, 0.23.0, 0.23.1, 0.23.2, 0.24.dev0, 0.24.0rc1, 0.24.0, 0.24.1, 0.24.2)
ERROR: No matching distribution found for scikit-learn==1.0.2 (from beep)
